## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-03-01-14-56-21 +0000,bbc,Nine dead in missile attack on Israel as Iran ...,https://www.bbc.com/news/articles/c363zkp1pgxo...
1,2026-03-01-14-56-20 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/03/01/world/...
2,2026-03-01-14-49-36 +0000,wapo,Airspace restrictions force major global hubs ...,https://www.washingtonpost.com
3,2026-03-01-14-49-36 +0000,wapo,Live updates: Casualties in Israel as Iran vow...,https://www.washingtonpost.com/world/2026/03/0...
4,2026-03-01-14-45-56 +0000,bbc,"The months-long, US-Israeli mission to find an...",https://www.bbc.com/news/articles/c86y5540vnno...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2541/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,iran,84
6,strikes,34
39,trump,29
77,iranian,21
4,israel,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
125,2026-03-01-01-01-13 +0000,wapo,"See where U.S., Israeli strikes have hit Iran ...",https://www.washingtonpost.com/investigations/...,236
53,2026-03-01-10-59-34 +0000,wapo,Iran’s supreme leader killed in U.S.-Israeli a...,https://www.washingtonpost.com/world/2026/02/2...,218
117,2026-03-01-01-52-12 +0000,nypost,Multiple members of Supreme Leader Ayatollah A...,https://nypost.com/2026/02/28/world-news/multi...,212
112,2026-03-01-02-19-45 +0000,bbc,Iran's Supreme Leader killed in joint US Israe...,https://www.bbc.com/news/articles/c70n9wlkx3lo...,205
70,2026-03-01-09-47-41 +0000,nyt,Israel launches new military strikes as Iran r...,https://www.nytimes.com/live/2026/03/01/world/...,179


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
125,236,2026-03-01-01-01-13 +0000,wapo,"See where U.S., Israeli strikes have hit Iran ...",https://www.washingtonpost.com/investigations/...
96,86,2026-03-01-04-56-41 +0000,nypost,Trump says diplomatic agreement with Iran will...,https://nypost.com/2026/02/28/world-news/trump...
163,74,2026-02-28-21-55-02 +0000,nyt,Israel Targeted Top Iranian Leaders in Attack’...,https://www.nytimes.com/2026/02/28/world/middl...
107,41,2026-03-01-02-53-00 +0000,wsj,Democrats in Congress led the charge to releas...,https://www.wsj.com/politics/policy/epstein-fi...
89,34,2026-03-01-05-44-12 +0000,nypost,Trump warns US will strike Iran with ‘force th...,https://nypost.com/2026/03/01/world-news/trump...
117,33,2026-03-01-01-52-12 +0000,nypost,Multiple members of Supreme Leader Ayatollah A...,https://nypost.com/2026/02/28/world-news/multi...
110,31,2026-03-01-02-21-55 +0000,nyt,The attacks touched off protests across the Un...,https://www.nytimes.com/live/2026/02/28/world/...
20,30,2026-03-01-13-44-15 +0000,cbc,"Hailing India as a 'natural partner,' Carney s...",https://www.cbc.ca/news/politics/india-natural...
176,27,2026-02-28-19-59-00 +0000,wsj,Glitzy Dubai Gets a Taste of Middle East War,https://www.wsj.com/world/middle-east/glitzy-d...
186,26,2026-02-28-18-40-55 +0000,nypost,Nancy Mace bashes Ilhan Omar over Iran attack ...,https://nypost.com/2026/02/28/us-news/nancy-ma...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
